In [ ]:
import os
import numpy as np
from PIL import Image
import pandas as pd


In [ ]:
# Directory paths for NDVI, SAVI, SDMI, Temperature, and Precipitation data
ndvi_folder = "C:\\Users\\Musae\\Documents\\GitHub-REPOs\\Senior-project_Doc\\Docs\\Array\\NDVI-Array"
savi_folder = "C:\\Users\\Musae\\Documents\\GitHub-REPOs\\Senior-project_Doc\\Docs\\Array\\SAVI-Array"
sdmi_folder = "C:\\Users\\Musae\\Documents\\GitHub-REPOs\\Senior-project_Doc\\Docs\\Array\\NDMI-Array"

In [ ]:
# Function to extract patches from an image using a generator to save memory
def extract_patches_generator(image, patch_size=(128, 128), step=128):
    img_height, img_width = image.shape[:2]
    patch_height, patch_width = patch_size
    
    for y in range(0, img_height, step):
        for x in range(0, img_width, step):
            patch = image[y:y+patch_height, x:x+patch_width]
            if patch.shape[0] == patch_height and patch.shape[1] == patch_width:
                yield patch.astype(np.float32)  # Convert patch to float32 to save memory

# Function to load an image or .npy file
def load_image(file_path):
    if file_path.endswith('.npy'):
        return np.load(file_path).astype(np.float32)  # Convert to float32 to save memory
    else:
        raise ValueError("Unsupported file format: {}".format(file_path))

# Function to process a folder of .npy files, yielding patches from each file to save memory
def process_folder_generator(folder_path, patch_size=(128, 128), step=128):
    file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)]

    for file_path in file_paths:
        image = load_image(file_path)
        for patch in extract_patches_generator(image, patch_size=patch_size, step=step):
            yield patch

# CSV data processing for temperature and precipitation (assuming already loaded into temperature_images and precipitation_images)
def climate_data_generator(climate_images, patch_size=(128, 128), step=128):
    for img in climate_images:
        for patch in extract_patches_generator(img, patch_size=patch_size, step=step):
            yield patch

In [ ]:
# Your CSV data processing for temperature and precipitation
csv_data = pd.read_csv("C:\\Users\\Musae\\Documents\\GitHub-REPOs\\Senior-project_Doc\\monthly_averages_formatted.csv")
image_height = 6345
image_width = 6445
temperature_images = []
precipitation_images = []

for index, row in csv_data.iterrows():
    temp_image = np.full((image_height, image_width), row['Temp Average'])
    precip_image = np.full((image_height, image_width), row['PRECTOTCORR Average'])
    temperature_images.append(temp_image)
    precipitation_images.append(precip_image)

In [ ]:
# Example usage of generators
ndvi_patches_generator = process_folder_generator(ndvi_folder)


In [ ]:
savi_patches_generator = process_folder_generator(savi_folder)


In [ ]:
sdmi_patches_generator = process_folder_generator(sdmi_folder)


In [ ]:
temperature_patches_generator = climate_data_generator(temperature_images)

In [ ]:
precipitation_patches_generator = climate_data_generator(precipitation_images)

In [ ]:
def combined_generator(*generators, batch_size=32):
    while True:
        batch = [next(gen) for gen in generators]  # Get next patch from each generator
        # Correctly stack patches to form a multi-channel input
        X_batch = np.stack(batch[:-1], axis=-1)  # Exclude label generator if used separately
        # Reshape to ensure batch dimension is included
        X_batch = np.reshape(X_batch, (batch_size, X_batch.shape[1], X_batch.shape[2], X_batch.shape[3]))
        y_batch = np.array(batch[-1])  # Assuming the last generator yields labels, adjust accordingly
        y_batch = y_batch[:batch_size]  # Ensure y_batch matches batch_size
        yield X_batch, y_batch


# CNN MODEL

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 4)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    # Decoder starts here
    Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu'),
    Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu'),
    Conv2D(1, (1, 1), activation='sigmoid', padding='same')  # Single output channel
])




In [ ]:
# Adjust the loss function and metrics based on your specific task
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Use 'mean_squared_error' for regression tasks

In [ ]:
def combined_generator(*generators, batch_size=32):
    while True:
        batch_X = []
        batch_y = []
        for _ in range(batch_size):  # Collect batches
            batch = [next(gen) for gen in generators]
            X = np.stack(batch[:-1], axis=-1)  # Stack input patches
            y = np.expand_dims(batch[-1], axis=-1)  # Expand dims of the last generator's output for labels
            batch_X.append(X)
            batch_y.append(y)
        
        # Stack along new axis to maintain batch dimension
        batch_X = np.stack(batch_X, axis=0)
        batch_y = np.stack(batch_y, axis=0)
        
        yield batch_X, batch_y


In [ ]:
# Test the combined_generator
test_combined_gen = combined_generator(ndvi_patches_generator, savi_patches_generator, sdmi_patches_generator, temperature_patches_generator, precipitation_patches_generator, batch_size=2)
X_batch_test, y_batch_test = next(test_combined_gen)
print(f"X_batch shape: {X_batch_test.shape}")
print(f"y_batch shape: {y_batch_test.shape}")


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam

In [ ]:
# # Use the adjusted combined_generator for model training
# combined_gen = combined_generator(ndvi_patches_generator, savi_patches_generator, sdmi_patches_generator, temperature_patches_generator, precipitation_patches_generator, batch_size=32)
# model.fit(combined_gen, steps_per_epoch=100, epochs=10)


# Define the optimizer with the desired learning rate
# Define the optimizer with the desired learning rate
opt = Adam(learning_rate=0.001)  # Adjust the learning rate as needed

# Compile the model with the specified optimizer
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])  # Adjust loss and metrics as needed

# Use the adjusted combined_generator for model training
combined_gen = combined_generator(ndvi_patches_generator, savi_patches_generator, sdmi_patches_generator, temperature_patches_generator, precipitation_patches_generator, batch_size=32)

# Train the model with the adjusted learning rate
history = model.fit(combined_gen, steps_per_epoch=100, epochs=10)
